In [1]:
from fastai.vision import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callbacks import SaveModelCallback

if torch.cuda.is_available():
  print("Great! Good to go!")
else:
  print('CUDA is not up!')

Great! Good to go!


In [2]:
from gepcore.utils import cell_graph
from gepnet.model_v2 import get_gepnet, arch_config
from gepnet.utils import count_parameters
torch.backends.cudnn.benchmark = True

In [3]:
from pygraphviz import AGraph
import glob

In [4]:
graph = [AGraph(g) for g in glob.glob('comp_graphs/experiment_6/best/indv_0/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)

In [5]:
import random
seed = 100
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# enable torch backends
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
#torch.backends.deterministic = True

In [7]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=36,
                   repeat_list=[5, 4, 2],
                   classes=10)

net = get_gepnet(conf)
count_parameters(net)

3.77221

In [ ]:
path = untar_data(URLs.CIFAR)
tfms = get_transforms(xtra_tfms=[cutout()]) 
bs = 128
data = (ImageList.from_folder(path/'train')
        .split_by_rand_pct(valid_pct=0.1, seed=seed) 
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize(cifar_stats))

In [ ]:
learn = Learner(data, net, metrics=[error_rate, accuracy], 
        model_dir='/home/cliff/ResearchProjects/models/evolution_search/').load('experiment_6_model_0').mixup()
learn.to_fp16()   

In [ ]:
learn.lr_find(wd=4e-4,end_lr=100)

learn.recorder.plot()

In [ ]:
cb = SaveModelCallback(learn, every='improvement', monitor='accuracy', name='experiment_6_model_0')
learn.fit_one_cycle(1000, 1e-3, wd=0.0004, callbacks=[cb]) 

In [ ]:
############# Testing #############

In [ ]:
graph = [AGraph(g) for g in glob.glob('comp_graphs/experiment_1/best/indv_0/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)

In [ ]:
conf = arch_config(comp_graph=comp_graph,
                   depth_coeff=1.0,
                   width_coeff=1.0,
                   channels=40,
                   repeat_list=[6, 6, 6, 6],
                   classes=10)

net = get_gepnet(conf)
count_parameters(net)

In [ ]:
tfms = get_transforms(do_flip=False)
path = untar_data(URLs.CIFAR)
bs = 256

data = (ImageList.from_folder(path)
        .split_by_folder(train='train', valid='test')
        .label_from_folder()
        .transform(tfms, size=32)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize(cifar_stats))

In [ ]:
model = Learner(data, net, metrics=[accuracy, error_rate]).load('experiment_1_model_1')
#model

In [ ]:
model.validate()

In [ ]:
preds,y,losses = model.get_preds(ds_type=DatasetType.Valid, with_loss=True)
preds[:5]